import urllib.request
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
import pymysql
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
import kss


#conn = pymysql.connect(host='newdb.c7p2ncpgik7h.ap-northeast-2.rds.amazonaws.com', user='admin', password='1dlckdals!',
#                       db='TEST1', charset='utf8')
#curs = conn.cursor()
#session = requests.Session()
#headers = {
#    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
#    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
#}


chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

#driver = webdriver.Chrome(r"/home/capston/chromedriver",chrome_options=chrome_options)
#driver = webdriver.Chrome(r"C:\Users\LCM\Downloads\chromedriver_win32 (2)\chromedriver.exe")
#driver = webdriver.Chrome(r"C:\Users\LCM\Downloads\chromedriver_win32\chromedriver.exe")
driver = webdriver.Chrome(r"C:\Users\seenw\Downloads\chromedriver_win32\chromedriver.exe")


class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        # 불용어 불러오기
        self.stopwords = [line.rstrip('\n') for line in open('stopwords_korean2.txt', encoding = 'utf-8')]
    
    def text2sentences(self, text):
        #sentences = self.kkma.sentences(text)\
        sentences = kss.split_sentences(text)
        sentences = sentences[0:len(sentences)-2]
        
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
                
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
                    if noun not in self.stopwords and len(noun) > 1]))
                
        return nouns


class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
    
    
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
                
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

    
    
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
            
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
        #print(self.nouns)
        
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
    
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
            
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
            
        return keywords


class newsCrawler:
    def __init__(self):
        self.titleList=[]
        self.contentsList=[]
        self.imageList=[]
        self.dateList=[]
        
    # 네이버 뉴스홈
    def mainCrawl(self):    
        # 정치=100 경제=101 사회=102 생활/문화=103 세계=104 IT/과학=105
        for category in range(100, 101):
            main_url = "https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1="+str(category)
            driver.get(main_url)
            
            # '헤드라인 더보기' 버튼이 있다면 누르기       
            self.showMore()
            driver.implicitly_wait(3)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')  
            
            # 모든 헤드라인 뉴스 가져오기
            articles = soup.find_all('div', {'class': 'cluster_group _cluster_content'})
        
            for i in range(len(articles)):
                # 각 뉴스 본문에 있는 이미지와 이미지URL를 저장할 리스트
                images=[]
                imagesURL="NO IMAGE"
                
                
                temp = articles[i].find_all('div', {'class': 'cluster_text'})[0]

                conURL = temp.a.get('href')
                html2 = session.get(conURL,headers=headers).content
                soup2 = BeautifulSoup(html2, 'html.parser')

                summary = soup2.find('strong', {'class':'media_end_summary'})
                if summary==None:
                    summary=""
                else:
                    summary=summary.text
            
                content = soup2.find('div', id= "articleBodyContents").text.replace("\n"," ").replace(str(summary),"").replace("\t"," ").replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}"," ")
                title=soup2.find('h3',id="articleTitle").text
            
                # 기사 본문이 10문장이하라면 저장하지 않는다.
                if(len(kss.split_sentences(content)) <= 10):
                    continue;
                
                #print(title)
                date=soup2.find('span', {'class','t11'}).text
                #print(contents)
                #print(date)
                
                # 이미지 추출
                images=soup2.find_all('span', {'class','end_photo_org'})
            
                for i in range(len(images)):
                    imagesURL=(images[i].find("img")["src"])
            
                self.saveToDB(str(title),str(content),str(imagesURL),str(date),str(category))


            # DB에 저장
            #self.saveToDB(self.titleList,self.contentsList,self.imageList,self.dateList)

            driver.quit()
    
    # 더보기버튼 클릭
    def showMore(self):
        try:
            while True:
                print("더보기")
                driver.find_element_by_xpath('//*[@id="main_content"]/div/div[2]/div[2]/div/a').click()
                driver.implicitly_wait(0.5)
        except exceptions.ElementNotVisibleException:
            pass
        except Exception:
            pass
    
    
    def saveToDB(self,title,content,imagesURL,date,category):
        content=content.replace("'","")
        sum = TextRank(content)
        
        # content로 
        
        
        content=sum.summarize(7)
        count=1
        # 요약문이 7문장보다 짧을 경우
        if len(content)<7:
            return 2
        
        for i in content:
            # 요약문에 빈문장이 포함된 경우
            if i=="":
                print('중지됨')
                return 3
            print(i)
            print(count)
            print("\n")
            count=count+1
        title = title.replace("'","")

#         # SQL문 실행
#         sql = "USE TEST1"
#         curs.execute(sql)
#         '''
#         CREATE TABLE NEWS4 (TITLE CHAR(200) NOT NULL,
#         CONTENT1 TEXT NOT NULL,
#         CONTENT2 TEXT NOT NULL,
#         CONTENT3 TEXT NOT NULL,
#         CONTENT4 TEXT NOT NULL,
#         CONTENT5 TEXT NOT NULL,
#         CONTENT6 TEXT NOT NULL,
#         CONTENT7 TEXT NOT NULL,
#         DATE VARCHAR(40) NOT NULL,
#         CATEGORY VARCHAR(40),
#         COUNT int NOT NULL AUTO_INCREMENT,
#         IMAGE TEXT NOT NULL,
#         CONSTRAINT PLAYER_PK PRIMARY KEY (COUNT));
#         '''

#         #sql3="insert into NEWS3(title,content,date,category,image) VALUES(" +title+ ',' +content+ ',' +date+ ',' +category+ ',' +imagesURL+ ");"
#         sql3="""insert into NEWS7(title,content1,content2,content3,content4,content5,content6,content7,date,category,image) VALUES('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s');"""%(title,content[0],content[1],content[2],content[3],content[4],content[5],content[6],date,category,imagesURL)
#         curs.execute(sql3)
#         conn.commit()
#         print("db updated!")

        return 1


Crawl=newsCrawler() 
Crawl.mainCrawl()
#conn.commit()
#curs.close()
print('done')

# 내코드

In [30]:
import urllib.request
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
import pandas as pd

import kss

class newsCrawler:
    def __init__(self):
        self.titleList=[]
        self.contentsList=[]
        self.imageList=[]
        self.dateList=[]
        
    # 네이버 뉴스홈
    def mainCrawl(self):    
        # 정치=100 경제=101 사회=102 생활/문화=103 세계=104 IT/과학=105
        # 실제 실행에서는 range(100, 106)으로 해야함.
        for category in range(101, 102):
            main_url = "https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1="+str(category)
            driver.get(main_url)
            
            # '헤드라인 더보기' 버튼이 있다면 누르기       
            self.showMore()
            driver.implicitly_wait(3)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')  
            
            # 헤드라인 가져오기
            self.subCrawl(soup, category)
        driver.quit()
    
    # 더보기버튼 클릭
    def showMore(self):
        try:
            while True:
                driver.find_element_by_xpath('//*[@id="main_content"]/div/div[2]/div[2]/div/a').click()
        except exceptions.ElementNotVisibleException as e:
            pass
        except Exception as e:
            pass
    
    # 헤드라인 뉴스 크롤링
    def subCrawl(self, soup, category):
        # 모든 헤드라인 뉴스 저장
        articles = soup.find_all('div', {'class': 'cluster_group _cluster_content'})
        
        for i in range(len(articles)):
            # 각 뉴스 본문에 있는 이미지와 이미지URL를 저장할 리스트
            images=[]
            imagesURL=[]
            
            # 본문 링크
            temp = articles[i].find_all('div', {'class': 'cluster_text'})[0]
            #print(temp.a.text)         
            conURL = temp.a.get('href')
            html2 = session.get(conURL,headers=headers).content
            soup2 = BeautifulSoup(html2, 'html.parser')
            
            # 기사 제목, 내용 추출
            # 기사 요약 부분있으면 본문에서 제외하기
            summary = soup2.find('strong', {'class':'media_end_summary'})
            if summary==None:
                summary=""
            else:
                summary=summary.text
            
            # 1. 본문 / 2. 제목
            contents = soup2.find('div', id= "articleBodyContents").text.replace("\n"," ").replace(str(summary),"").replace("\t"," ").replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}"," ")
            title=soup2.find('h3',id="articleTitle").text
            
            # 기사 본문이 10문장이하라면 저장하지 않는다.
            if(len(kss.split_sentences(contents)) <= 10):
                continue;
            
            # 3. 날짜
            date=soup2.find('span', {'class','t11'}).text
            self.dateList.append(date)
            
            images=soup2.find_all('span', {'class','end_photo_org'})
            # 5. 이미지url
            for i in range(len(images)):
                imagesURL.append(images[i].find("img")["src"])
                #print(imagesURL[i])
            
            # 기사 제목과 내용, 이미지URL 각 리스트에 저장
            self.titleList.append(title)
            self.contentsList.append(contents)
            self.imageList.append(imagesURL)   
        
#         sum_List=[]
#         for i in range(len(self.titleList)):
#             content_sum = TextRank(self.contentsList[i])
#             content_sum = content_sum.summarize(7)
#             sum_List.append(content_sum)
#         print(content_sum)
        
#         news_sum_df = pd.DataFrame(content_sum, columns=self.titleList)
#         print(news_sum_df)
        
        # 크롤링한 기사 정보로 데이터프레임 생성
        news_df = pd.DataFrame({'title':self.titleList, 'content':self.contentsList, 'date':self.dateList})
        news_df['category'] = pd.Series([category for i in range(len(news_df.index))])
        print(news_df)
        news_df.to_excel('news.xlsx')
        
        # 파일 저장
        self.saveNewsToFile()
        
        # db에 저장
        #self.saveToDB(news_df)
    
    def saveNewsToFile(self):
        f = open('result0319.txt', 'w', encoding='utf-8')
        for i in range(len(self.titleList)):
            f.write(self.titleList[i] + "\n")
            f.writelines(self.contentsList[i]+"\n")
            f.writelines(self.imageList[i])
            f.writelines("\n\n")
        f.close()
    
    def saveToDB(self, df):

#         # SQL문 실행
#         sql = "USE TEST1"
#         curs.execute(sql)
#         '''
#         CREATE TABLE NEWS4 (TITLE CHAR(200) NOT NULL,
#         CONTENT1 TEXT NOT NULL,
#         CONTENT2 TEXT NOT NULL,
#         CONTENT3 TEXT NOT NULL,
#         CONTENT4 TEXT NOT NULL,
#         CONTENT5 TEXT NOT NULL,
#         CONTENT6 TEXT NOT NULL,
#         CONTENT7 TEXT NOT NULL,
#         DATE VARCHAR(40) NOT NULL,
#         CATEGORY VARCHAR(40),
#         COUNT int NOT NULL AUTO_INCREMENT,
#         IMAGE TEXT NOT NULL,
#         CONSTRAINT PLAYER_PK PRIMARY KEY (COUNT));
#         '''

#         #sql3="insert into NEWS3(title,content,date,category,image) VALUES(" +title+ ',' +content+ ',' +date+ ',' +category+ ',' +imagesURL+ ");"
#         sql3="""insert into NEWS7(title,content1,content2,content3,content4,content5,content6,content7,date,category,image) VALUES('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s');"""%(title,content[0],content[1],content[2],content[3],content[4],content[5],content[6],date,category,imagesURL)
#         curs.execute(sql3)
#         conn.commit()
#         print("db updated!")

        return 1
    
    def sim_cosine(self):
        f = open('news.xlsx', 'a+', encoding = 'utf-8')

session = requests.Session()

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}

#driver = webdriver.Chrome(r"C:\Users\LCM\Downloads\chromedriver_win32\chromedriver.exe")
driver = webdriver.Chrome(r"C:\Users\seenw\Downloads\chromedriver_win32\chromedriver.exe")

c=newsCrawler()
c.mainCrawl()

                                title  \
0     바이비트, 6개월만에 비트코인 선물거래소 세계 2위 등극   
1   '美금리 부담' 코스피 1% 하락 3030선…개인 홀로 사자   
2  LG·SK '배터리 특허침해' 미국 ITC 예비판결 2주 연기   
3    삼성물산, 첫 사외이사 이사회 의장 선임…ESG 경영 강화   
4   '저세상 매운맛' 눈 뒤집힌 해외…불닭볶음면 3000억 대박   
5              그린북서 ‘불확실성’ 표현 삭제한 기재부   

                                             content                  date  \
0             비트코인 선물거래소 순위에서 6위에 머물던 바이비트가 불과 6개...   2021.03.18. 오후 3:48   
1          15일 오후 서울 중구 하나은행 딜링룸 전광판. 2021.3.15/뉴...   2021.03.19. 오전 9:52   
2          LG에너지솔루션이 SK이노베이션을 상대로 미국 국제무역위원회에 제기한...   2021.03.19. 오전 9:31   
3          뉴스1 © News1 구윤성 기자(서울=뉴스1) 이훈철 기자 = 정병...   2021.03.19. 오후 2:23   
4          삼양식품이 매운맛으로 특히 동남아에서 인기가 높은 ‘불닭볶음면’을 앞...  2021.03.19. 오전 10:37   
5          정부가 경기평가에서 10개월만에 ‘불확실성’이라는 표현을 삭제했다. ...  2021.03.19. 오전 11:39   

   category  
0       101  
1       101  
2       101  
3       101  
4       101  
5       101  


In [13]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

import kss


class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        # 불용어 불러오기
        self.stopwords = [line.rstrip('\n') for line in open('stopwords_korean3.txt', encoding = 'utf-8')]
    
    def text2sentences(self, text):
        #sentences = self.kkma.sentences(text)\
        sentences = kss.split_sentences(text)
        # 기사의 마지막 문장 1개 제외시킴
        sentences = sentences[0:len(sentences)-2]
        
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
                
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
                    if noun not in self.stopwords and len(noun) > 1]))
                
        return nouns


class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        # tfidf_mat의 크기는 문장수x단어수
#         print("tf-idf 행렬:")
#         print(tfidf_mat)
#         print(tfidf_mat.shape)
        
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
#         print("graph_sentence 행렬:")
#         print(self.graph_sentence)
#         print(self.graph_sentence.shape)
        # graph_sentence는 각 문장 벡터들간의 유사도 행렬이므로
        # 행렬의 크기는 문장수x문장수
        # 그래프로 그리면 노드가 문장수만큼 존재하고,
        # 이 그래프를 textrank 알고리즘의 입력으로 사용하여 각 문장의 점수를 구한다.
        # 점수가 높은 상위 n개를 선택하여 문서의 요약문으로 삼는다.
        
        
        #print(self.tfidf.get_feature_names())
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
    
    
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
                
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

    
    
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
            
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        #print(self.sent_graph)
        #print(self.sent_graph.shape)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
        #print(self.nouns)
        
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
    
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
            
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
            
        return keywords

# db와 연결 - db에 있는 기사 본문?요약?들을 tf-idf행렬로 벡터화 - 각 유사도 계산
# - 새로 크롤링한 기사들과 유사한 '최근' 기사 5개씩 db에 저장

# 아니면 db와 연결하지 말고 txt파일로 저장하는걸 추가로 구현한 다음에
# txt를 읽고 유사도를 계산하는게 더 나을수도

# 그럼 txt파일 or csv파일에 어떤 내용들을 저장할까
# 지금까지 크롤링한 기사본문/카테고리(100~105)/tf-idf행렬

# class Recommendation(object):
#     def __init__(self, text):
        
#     def cosine_sim(self):
        
#     def get_recommendation(self):
        

In [5]:
f = open('result0319.txt', 'r', encoding='utf-8')

summary = []
count=1

while True:
    line = f.readline()
    # text파일에서 본문만 읽음
    if count%4 == 2:
        sum = TextRank(line)
        #print(sum.summarize(8)) # 길이가 n인 리스트 출력
        #print(len(sum.summarize(8)))
        for row in sum.summarize(6):
            summary.append(row)
            print(row)
            print()
        print()
        print()
        
    if not line:
        break
    
    count+=1

f.close()

tf-idf 행렬:
[[0.         0.23810353 0.         ... 0.21185339 0.         0.19149219]
 [0.         0.         0.         ... 0.24419311 0.         0.22072375]
 [0.417061   0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.24797927 0.28651131 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(14, 76)
graph_sentence 행렬:
[[1.         0.17150872 0.08594865 0.         0.0681443  0.
  0.20064288 0.18776911 0.         0.         0.         0.
  0.16726781 0.        ]
 [0.17150872 1.         0.         0.04686927 0.13392548 0.11929749
  0.18253282 0.3961348  0.08015994 0.         0.02946523 0.10462854
  0.22383751 0.08916593]
 [0.08594865 0.         1.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.04686927 0.         1.         0.07556445 0.
  0.  

tf-idf 행렬:
[[0.         0.         0.         ... 0.         0.30988344 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.31031755 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.41938104 0.        ]]
(16, 149)
graph_sentence 행렬:
[[1.         0.21500633 0.1149017  0.0917091  0.         0.4519745
  0.08617499 0.         0.17402207 0.04022795 0.04349686 0.13679924
  0.07437969 0.45822071 0.04607436 0.40133333]
 [0.21500633 1.         0.18753853 0.05944076 0.         0.20186543
  0.07041967 0.05474334 0.14430448 0.03689542 0.         0.19290526
  0.06411247 0.02294471 0.05277797 0.12545785]
 [0.1149017  0.18753853 1.         0.25664201 0.04843646 0.30466472
  0.03232425 0.04783562 0.04113009 0.         0.16225636 0.46005086
  0.11495939 0.

tf-idf 행렬:
[[0.         0.         0.         ... 0.         0.34531026 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.28291684]
 ...
 [0.         0.         0.39549427 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.26418028 0.         ... 0.26418028 0.         0.        ]]
(13, 86)
graph_sentence 행렬:
[[1.         0.38713398 0.18289512 0.15535888 0.1285096  0.06907253
  0.48642373 0.14168277 0.31096673 0.09497187 0.0622722  0.
  0.        ]
 [0.38713398 1.         0.19722527 0.26349117 0.09442389 0.05075183
  0.42528919 0.24430557 0.11251968 0.12891502 0.04575521 0.
  0.        ]
 [0.18289512 0.19722527 1.         0.         0.04539889 0.09229767
  0.28442013 0.184871   0.11282238 0.12773402 0.04399809 0.
  0.        ]
 [0.15535888 0.26349117 0.         1.         0.2244378  0.12993115
  0.         0.10224929

tf-idf 행렬:
[[0.         0.18483799 0.         ... 0.         0.12983927 0.        ]
 [0.18634231 0.         0.18634231 ... 0.         0.30148847 0.        ]
 [0.         0.         0.         ... 0.         0.25631249 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.26711143 0.         0.        ]]
(15, 141)
graph_sentence 행렬:
[[1.         0.11972298 0.03327943 0.33729722 0.04596937 0.03907808
  0.02020284 0.03645186 0.12255523 0.         0.0427392  0.
  0.08417337 0.         0.02584233]
 [0.11972298 1.         0.1953568  0.16751557 0.06534257 0.
  0.01172781 0.08464168 0.         0.         0.08173709 0.06691185
  0.01483171 0.         0.01500156]
 [0.03327943 0.1953568  1.         0.05929659 0.11859396 0.11787956
  0.         0.         0.         0.15211401 0.         0.
  0.         0.         0.        ]
 [0.33729722 0.16751557 

In [5]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
import kss

okt = Okt()

text2='''[파이낸셜뉴스] 현대차가 프리미엄 다목적 차량(MPV) 스타리아의 티저 이미지를 공개했다.   11일 현대차에 따르면 스타리아는 프리미엄 멀티 밴으로 맞춤형 이동수단(PBV)과 가장 유사한 공간성을 경험할 수 있는 차종이다. 명칭인 스타리아는 별(STAR)과 물결(RIA)의 합성어로 별 사이를 유영하는 우주선 외관에서 영감을 받아 차명으로 결정됐다.   이번에 공개한 티저 이미지는 고급 모델인 '스타리아 라운지' 7인승으로 볼륨감 있는 외관과 개방감을 극대화한 실내 디자인이 특징이다. 높은 전고와 긴 전폭 및 전장으로 공간감을 극대화했다. 낮은 벨트라인을 통한 실내 개방감과 혁신적인 디자인으로 고객에게 새로운 이동 경험을 제공한다.   전면부는 크고 넓은 라디에이터 그릴과 차체를 가로지르는 얇고 긴 주간주행등(DRL), 낮게 위치한 헤드램프로 하이테크한 이미지를 연출했다. 차량 전면부터 후면까지 이어지는 유려한 곡선은 우주선을 연상시키며 픽셀 타입의 리어 콤비램프는 고급스러움을 더한다.   크루즈의 라운지에서 영감을 얻은 실내 공간은 고급스러움과 여유로운 공간을 갖췄다. 간결함과 개방감이 뛰어난 운전석과 조수석을 포함해 실내 공간은 '인사이드 아웃' 디자인이 적용돼 승객의 편의성과 사용성을 극대화했다.   현대디자인담당 이상엽 전무는 "스타리아는 새로운 모빌리티 시대를 여는 첫 MPV"라며 "차별화된 디자인을 통해 고객에게 새로운 경험과 가치를 제공할 것"이라고 말했다.    스타리아 티저 이미지  스타리아 티저 이미지 cynical73@fnnews.com 김병덕 기자   ▶ 헉! 소리나는 스!토리 뉴스 [헉스]▶ '아는 척'하고 싶은 당신을 위한 [두유노우]▶ 날로먹고 구워먹는 금융이슈 [파인애플]  ※ 저작권자 ⓒ 파이낸셜뉴스. 무단 전재-재배포 금지'''
i=0
sentences = kss.split_sentences(text2)
for sent in kss.split_sentences(text2):
    print(sent)
    print(i)
    i+=1

sentences = sentences[0:len(sentences)-2]
stopwords = [line.rstrip('\n') for line in open('stopwords_korean3.txt', encoding = 'utf-8')]
nouns = []
for sentence in sentences:
    if sentence != '':
        nouns.append(' '.join([noun for noun in okt.nouns(str(sentences))
            if noun not in stopwords and len(noun) > 1]))
print(nouns)
print(len(nouns))

[파이낸셜뉴스] 현대차가 프리미엄 다목적 차량(MPV) 스타리아의 티저 이미지를 공개했다.
0
11일 현대차에 따르면 스타리아는 프리미엄 멀티 밴으로 맞춤형 이동수단(PBV)과 가장 유사한 공간성을 경험할 수 있는 차종이다.
1
명칭인 스타리아는 별(STAR)과 물결(RIA)의 합성어로 별 사이를 유영하는 우주선 외관에서 영감을 받아 차명으로 결정됐다.
2
이번에 공개한 티저 이미지는 고급 모델인 '스타리아 라운지' 7인승으로 볼륨감 있는 외관과 개방감을 극대화한 실내 디자인이 특징이다.
3
높은 전고와 긴 전폭 및 전장으로 공간감을 극대화했다.
4
낮은 벨트라인을 통한 실내 개방감과 혁신적인 디자인으로 고객에게 새로운 이동 경험을 제공한다.
5
전면부는 크고 넓은 라디에이터 그릴과 차체를 가로지르는 얇고 긴 주간주행등(DRL), 낮게 위치한 헤드램프로 하이테크한 이미지를 연출했다.
6
차량 전면부터 후면까지 이어지는 유려한 곡선은 우주선을 연상시키며 픽셀 타입의 리어 콤비램프는 고급스러움을 더한다.
7
크루즈의 라운지에서 영감을 얻은 실내 공간은 고급스러움과 여유로운 공간을 갖췄다.
8
간결함과 개방감이 뛰어난 운전석과 조수석을 포함해 실내 공간은 '인사이드 아웃' 디자인이 적용돼 승객의 편의성과 사용성을 극대화했다.
9
현대디자인담당 이상엽 전무는 "스타리아는 새로운 모빌리티 시대를 여는 첫 MPV"라며 "차별화된 디자인을 통해 고객에게 새로운 경험과 가치를 제공할 것"이라고 말했다.
10
스타리아 티저 이미지  스타리아 티저 이미지 cynical73@fnnews.com 김병덕 기자   ▶ 헉! 소리나는 스!토리 뉴스 [헉스]▶ '아는 척'하고 싶은 당신을 위한 [두유노우]▶ 날로먹고 구워먹는 금융이슈 [파인애플]  ※ 저작권자 ⓒ 파이낸셜뉴스. 무단 전재-재배포 금지
11
['현대차 프리미엄 목적 차량 스타 리아 티저 이미지 공개 현대차 스타 리아 프리미엄 멀티 맞춤 이동 수단 가장 공간 경험 종이 명칭 스타 리아 물결 합성어 사이 유영 우주선 외관 영감

In [76]:
import pandas as pd


lt = ['제목','본문','날짜','카테고리']
title = ['a','b','c','d']
content = ['e','f','g','h']
title2 = ['11','22','33','44']
content2 = ['5','6','7','8']
summary = ['1','2','3','4']
summ = [[1,2,3],[4,5,6]]
df = pd.DataFrame({'title':title, 'content':content})
print(df)


df.set_index('title', inplace=True)

#df.append({'sum':summary}, ignore_index=True)
#df.append(summ)

# print(df['content'])

# doc = list(df['content'])
# print(type(doc))
# print(doc)

df2 = pd.DataFrame({'title':title2, 'content':content2})

df2.set_index('title', inplace = True)

df = pd.concat([df, df2])
print(df)

  title content
0     a       e
1     b       f
2     c       g
3     d       h
      content
title        
a           e
b           f
c           g
d           h
11          5
22          6
33          7
44          8
